Ambiente miniconda modeller

In [2]:
from modeller import *
from modeller.automodel import *
import os.path
from pathlib import Path
from Bio import SeqIO
from Bio.PDB.PDBIO import Select
from Bio.PDB import PDBParser, PPBuilder
from Bio.PDB import PDBIO
from io import StringIO
import os
import subprocess
import requests
import urllib
import sys
import urllib.request

In [ ]:
#=== Descarga de archivo PDB (templado) ===

template = '4zwb'
uniprotcd= 'P11166'

def download_pdb(pdbid, url="https://files.rcsb.org/download/"):
    targetdir = "../run/modelamineto_dinamica"
    os.makedirs(targetdir, exist_ok=True)
    
    pdbfullname = pdbid + ".pdb"
    # Modificar outfnm para incluir la ruta de destino
    out = os.path.join(targetdir, pdbfullname)

    if os.path.exists(out):
        print(f"{pdbfullname} ya existe. Abortando descarga.")
        return out
    else:
        url = url + pdbfullname
        try:
            urllib.request.urlretrieve(url, out)
            print(f"Archivo descargado en {targetdir}")
            return out
        except Exception as err:
            print(f"Error al descargar el archivo {pdbfullname}: {err}", file=sys.stderr)
            return None
            
download_pdb(template)

4zwb.pdb ya existe. Abortando descarga.


'../run/modelamineto_dinamica/4zwb.pdb'

In [4]:
#=== Limpieza de prefijos y residuos no deseados en un archivo PDB ===

class CustomPDBSelector(Select):
    """
    Permite seleccionar una cadena específica y excluir residuos HETATM que no sean agua (HOH).
    """
    def __init__(self, chain_id):
        self.chain_id = chain_id

    def accept_chain(self, chain):
    #   Acepta solo la cadena con el ID especificado.
        return chain.id == self.chain_id

    def accept_residue(self, residue):
    #    Acepta residuos de tipo ATOM y residuos HETATM que sean agua (HOH).
    #    Excluye otros residuos HETATM.
    
        # Si es un residuo HETATM y no es HOH (agua), no lo acepta.
        if residue.id[0] != ' ' and residue.get_resname().strip() != 'HOH':
            return False
        return True # Acepta otros residuos (ATOM o HOH)

def clean_pdb(template, chain_id='A'):
    """
    Limpia un archivo PDB:
    1. Selecciona una cadena específica.
    2. Elimina residuos HETATM que no sean agua.
    3. Filtra líneas que comienzan con prefijos no deseados (ANISOU, HET, HETNAM, HETSYN, FORMUL, CONECT).
    """
    pdb_file = f"../run/modelamineto_dinamica/{template}.pdb"
    clean_file = f"../run/modelamineto_dinamica/{template}_clean.pdb"

    # 1. Parsear el archivo PDB
    parser = PDBParser(QUIET=True)
    try:
        structure = parser.get_structure('structure', pdb_file)
    except Exception as e:
        print(f"Error al parsear el archivo PDB {pdb_file}: {e}", file=sys.stderr)
        return None

    # Verificar si la estructura o el modelo existen
    if not structure or not structure[0]:
        print(f"No se encontraron modelos o estructura en el archivo PDB {pdb_file}", file=sys.stderr)
        return None
    
    # Verificar si la cadena especificada existe en el primer modelo
    target_model = structure[0]
    chain_ids = [chain.id for chain in target_model]
    if chain_id not in chain_ids:
        print(f"La cadena '{chain_id}' no se encontró en la estructura. Cadenas disponibles: {chain_ids}", file=sys.stderr)
        return None
    
    # 2. Seleccionar la cadena y excluir HETATM (no-agua), escribiendo a un buffer en memoria
    io = PDBIO()
    io.set_structure(structure) # Se establece la estructura completa en PDBIO
    
    # Usar StringIO para capturar la salida de PDBIO en memoria en lugar de un archivo en disco
    pdb_string_io = StringIO()
    io.save(pdb_string_io, CustomPDBSelector(chain_id)) # Se pasa el selector personalizado
    pdb_string_io.seek(0) # Rebobinar el buffer al inicio para poder leerlo
    
    # 3. Limpiar líneas adicionales no deseadas del buffer en memoria y escribir al archivo final
    
    exclude_prefixes_line_filter = ['ANISOU', 'HET', 'HETNAM', 'HETSYN', 'FORMUL', 'CONECT']
    
    with open(clean_file, 'w') as f_out:
        for line in pdb_string_io:
            # Escribir la línea solo si no comienza con ninguno de los prefijos a excluir
            if not any(line.startswith(prefix) for prefix in exclude_prefixes_line_filter):
                f_out.write(line)

    print(f"Archivo PDB limpio guardado como: {clean_file}")
    return clean_file

clean_pdb(template, chain_id='A')

Archivo PDB limpio guardado como: ../run/modelamineto_dinamica/4zwb_clean.pdb


'../run/modelamineto_dinamica/4zwb_clean.pdb'

In [5]:
parser = PDBParser(QUIET=True)
structure = parser.get_structure("protein", f'../run/modelamineto_dinamica/{template}_clean.pdb')
ppb = PPBuilder()

chain_id = 'A'
full_sequence = ""

for model in structure:
    for chain in model:
        if chain.id == chain_id:
            for pp in ppb.build_peptides(chain):
                frag_seq = pp.get_sequence()
                print(f"Fragment for chain {chain.id}:")
                print(frag_seq)
                full_sequence += str(frag_seq)

fasta_template = f"../run/modelamineto_dinamica/{template}.fasta"
with open(fasta_template, "w") as f:
    f.write(f">{template}\n")
    f.write(full_sequence + "\n")

Fragment for chain A:
TQKVTPALIFAITVATIGSFQFGYNTGVINAPEKIIKEFITKTLTDKGNAPPSEVLLTSLWSLSVAIFSVGGMIGSFSVGLFVNRFGRRNSMLIVNLLAVTGGCFMGLCKVAKSVEMLILGRLVIGLFCGLCTGFVPMYIGEISPTALRGAFGTLNQLGIVVGILVAQIFGLEFILGSEELWPLLLGFTILPAILQSAALPFCPESPRFLLINRKEEENAKQILQRLWGTQDVSQDIQEMKDESARMSQEKQVTVLELFRVSSYRQPIIISIVLQLSQQLSGINAVFYYSTGIFKDAGVQEPIYATIGAGVVNTIFTVVSLFLVERAGRRTLHMIGLGGMAFCSTLMTVSLLLKDNYNGMSFVCIGAILVFVAFFEIGPGPIPWFIVAELFSQGPRPAAMAVAGCSNWTSNFLVGLLFPSAAHYLGAYVFIIFTGFLITFLAFTFFKVPETRGRTFEDITRAFEGQAH


In [16]:
#=== Concatenar archivos fasta ===
fastaP11166 = f'../data/GTR1_HUMAN.fasta'
output = '../run/modelamineto_dinamica/alignment.fasta'

with open(output, 'w') as out:
    for file in [fasta_template, fastaP11166]:
        with open(file, 'r') as f:
            out.write(f.read())

print(f'Archivo guardado en {output}')

#=== Alineamiento MAFFT ===
mafft_cmd = ["mafft", "--auto", output]
try:
    alignment = subprocess.check_output(mafft_cmd, universal_newlines=True)
    print("Alignment successful")

    # Save result
    with open("../run/modelamineto_dinamica/outputmft.fasta", "w") as ali_file:
        ali_file.write(alignment)
    print("Alignment saved to mafft_output.fasta")

except subprocess.CalledProcessError as e:
    print("Alignment failed:", e)

Archivo guardado en ../run/modelamineto_dinamica/alignment.fasta
Alignment successful
Alignment saved to mafft_output.fasta


outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.505
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
    0 / 2
done.

Progressive alignment ... 
STEP     1 /1 
done.
tbfast (aa) Version 7.505
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
rescale = 1

    0 / 2
Segment   1/  1    1- 493
done 001-001-1  identical.   
dvtditr (aa) Version 7.505
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 L-INS-i (Probably most accurate, very slow)
 Iterative refinement me

In [17]:
# Read MAFFT output file
with open("../run/modelamineto_dinamica/outputmft.fasta", "r") as file:
    lines = file.readlines()

# Split sequences
records = {}
current_id = None
for line in lines:
    line = line.strip()
    if line.startswith(">"):
        current_id = line[1:]
        records[current_id] = ""
    else:
        records[current_id] += line

# Now format as PIR
if len(records) != 2:
    print("Expected exactly 2 sequences for PIR format.")
else:
    seq_id, template_id = list(records.keys())
    seq_seq = records[seq_id]
    template_seq = records[template_id]

    pir_lines = [
        f">P1;{uniprotcd}",
        f"sequence:{uniprotcd}::A::A::::",
        template_seq + "*",
        f">P1;{seq_id}",
        f"structureX:{seq_id}::A::A::::",
        seq_seq + "*"
    ]

    with open("../run/modelamineto_dinamica/input_mdller.ali", "w") as f:
        f.write("\n".join(pir_lines))
        print("done")

done


In [29]:
def autoModel_modeller():
    alignments = []
    alignments.append("../run/modelamineto_dinamica/input_mdller.ali")

    print(f'Inputs: {alignments}, {template}, secuencia de {uniprotcd}')
    
    for align in alignments:
        env = Environ()
        env.io.atom_files_directory = ['./', '../run/modelamineto_dinamica/']
        a = AutoModel(env, alnfile=align,
                knowns=f'{template}', sequence=uniprotcd,
                assess_methods=(assess.DOPE,
                                assess.GA341),)
        a.starting_model = 1
        a.ending_model = 5
        a.make()
        alignments = []

autoModel_modeller()

Inputs: ['../run/modelamineto_dinamica/input_mdller.ali'], 4zwb, secuencia de P11166

check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.
              Total number of restraints before, now:    49272    45808
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_397

Ambiente miniconda glutoso (ambertools / openMM)

In [ ]:
#=== Ejecutar de manera local ===
packmol-memgen -p P11166.pdb --log log_file --salt --salt_c Na+ --saltcon 0.15 \
-l POPC:POPE:POPI:POPS:POPA:SSM:CHL1//POPC:POPE:SSM:CHL1 -r 18:26:5:11:1:10:29//38:6:22:37 \
--ffprot ff19SB --fflip lipid21 --ffwat tip3p --dims 140 140 140 --pdb2pqr --pdb2pqr_pH 7.4 --nottrim

In [ ]:
#=== Simulación de dinámica molecular 400 ps ===
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout
# load AMBER format files
prmtop = app.AmberPrmtopFile('bilayer_P11166_DB01914_lipid.top')
inpcrd = app.AmberInpcrdFile('bilayer_P11166_DB01914_lipid.crd')

system = prmtop.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer,
        constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)

# create simulation instance.
platform = mm.Platform.getPlatform('OpenCL')

simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

simulation.minimizeEnergy()

#Warm up systems.  Slowly warm up temperature - every 1000 steps raise the temperature by 5 K, ending at 300 K
simulation.reporters.append(
    app.StateDataReporter(
        stdout,
        5000,
        step=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
        density=True
    )
)
simulation.context.setVelocitiesToTemperature(5*unit.kelvin)
print('Warming up the system...')
T = 5
mdsteps = 50000
stages = 60
steps_per_stage = int(mdsteps//stages)
for i in range(60):
    temperature = (T+(i*T))*unit.kelvin 
    if i%20 == 0:
        print(f"heating to {temperature}")
    integrator.setTemperature(temperature)
    simulation.step(steps_per_stage)
# give some extra steps to maintain temperature equilibrium
print(f"Now it's around 300K")
simulation.step(5001)

#NPT equilibration, reducing backbone constraints
mdsteps = 50000
barostat = system.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))
simulation.context.reinitialize(True)
print('Running NPT equilibration...')
simulation.step(mdsteps)
simulation.saveState("equil.xml")

simulation.reporters[-1] = app.StateDataReporter(
        stdout,
        5000,
        step=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
        density=True,
        progress=True,
        remainingTime=True,
        speed=True,
        totalSteps=mdsteps,
        separator="\t"
    )
simulation.reporters.append(
    app.DCDReporter(('10ns_traj.dcd'), 5000))
mdsteps = 100000
print('Running Production...')
simulation.step(mdsteps)
print('Done!')

simulation.saveState("prod.xml")